# Lesson 1 - Building a QA Agent with Google Gemini

```mermaid
graph TD
    %% User / Client Layer
    User([User / A2A Client])
    
    %% Main Orchestrator Layer (Lesson 8)
    subgraph OrchestratorLayer [Router/Requirement Agent]
        Concierge["<b>Healthcare Concierge Agent</b><br/>(BeeAI Framework)<br/><code>Port: 9996</code>"]
    end

    subgraph SubAgents [A2A Agent Servers]
        direction LR

        PolicyAgent["<b>Policy Agent</b><br/>(Gemini with A2A SDK)<br/><code>Port: 9999</code>"]
        ResearchAgent["<b>Research Agent</b><br/>(Google ADK)<br/><code>Port: 9998</code>"]

        ProviderAgent["<b>Provider Agent</b><br/>(LangGraph + LangChain)<br/><code>Port: 9997</code>"]
    end

    %% Data & Tools Layer
    subgraph DataLayer [Data Sources & Tools]
        PDF["Policy PDF"]
        Google[Google Search Tool]
        MCPServer["FastMCP Server<br/>(<code>doctors.json</code>)"]
    end
    
    Label_UA["Sends Query - A2A"]
    Label_CP["A2A"]
    Label_CR["A2A"]
    Label_CProv["A2A"]
    Label_MCP["MCP (stdio)"]

    %% -- CONNECTIONS --
    
    User --- Label_UA --> Concierge

    Concierge --- Label_CP --> PolicyAgent
    Concierge --- Label_CR --> ResearchAgent
    Concierge --- Label_CProv --> ProviderAgent
    
    PolicyAgent -- "Reads" --> PDF
    ResearchAgent -- "Calls" --> Google
    
    ProviderAgent --- Label_MCP --> MCPServer

    classDef orchestrator fill:#f9f,stroke:#333,stroke-width:2px;
    classDef agent fill:#e1f5fe,stroke:#0277bd,stroke-width:2px;
    classDef tool fill:#fff3e0,stroke:#ef6c00,stroke-width:1px,stroke-dasharray: 5 5;
    
    classDef protocolLabel fill:#ffffff,stroke:none,color:#000;
    
    class Concierge orchestrator;
    class PolicyAgent,ResearchAgent,ProviderAgent agent;
    class PDF,Google,MCPServer tool;
    
    class Label_UA,Label_CP,Label_CR,Label_CProv,Label_MCP protocolLabel;
```

In this lesson, you will build a basic Question Answering (QA) agent using Google's Gemini model via the Google Gemini API. You will use **LiteLLM** to interact with the model to analyze a PDF document containing health insurance policy details. Then, you will refactor this logic into a reusable Python class, laying the groundwork for the next lesson where you will wrap this agent in an Agent2Agent (A2A) server.

<p style="background-color:#fff6ff; padding:15px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px"> 💻 &nbsp; <b>To Access <code>requirements.txt</code> and the <code>data</code> files:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Open"</em>. For more help, please see the <em>"Appendix – Tips, Help, and Download"</em> Lesson.</p>

## 1.1. Import Libraries and Setup

First, import the necessary libraries. You will use `litellm` to interact with the LLM and standard libraries to handle file encoding.

In [ ]:
import base64
from pathlib import Path

from IPython.display import Markdown, display
import litellm
from dotenv import load_dotenv

In [ ]:
_ = load_dotenv()

## 1.2. Load and Encode Data

You read the insurance policy PDF (`2026AnthemgHIPSBC.pdf`) and encode it in base64 so it can be passed to the model as context.


In [ ]:
with Path("../data/2026AnthemgHIPSBC.pdf").open("rb") as file:
    pdf_data = base64.standard_b64encode(file.read()).decode("utf-8")

## 1.3. Query the Model

Now you will send a specific query to the model: "How much would I pay for mental health therapy?". You provide the model with a system instruction to act as an expert insurance agent and pass the PDF document alongside the user's text prompt.


In [ ]:
prompt = "How much would I pay for mental health therapy?"

In [ ]:
response = litellm.completion(
    model="gemini/gemini-2.0-flash",
    messages=[
        {
            "role": "system",
            "content": "You are an expert insurance agent designed to assist with coverage queries. Use the provided documents to answer questions about insurance policies. If the information is not available in the documents, respond with 'I don't know'"
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:application/pdf;base64,{pdf_data}"
                    }
                }
            ]
        }
    ],
)

In [ ]:
response_text = response.choices[0].message.content.replace("$", r"\\$")
display(Markdown(response_text))

## 1.4. Refactor into an Agent Class

To make this code reusable and easier to integrate into an A2A server later, you will wrap the logic into a `PolicyAgent` class in a file named `policy_agent.py`. This class initializes the data in the `__init__` method and exposes an `answer_query` method.

In [ ]:
%%writefile policy_agent.py
import base64
from pathlib import Path

import litellm
from dotenv import load_dotenv

class PolicyAgent:
    def __init__(self) -> None:
        load_dotenv()
        with Path("../data/2026AnthemgHIPSBC.pdf").open("rb") as file:
            self.pdf_data = base64.standard_b64encode(file.read()).decode("utf-8")

    def answer_query(self, prompt: str) -> str:
        response = litellm.completion(
            model="gemini/gemini-2.0-flash",
            messages=[
                {
                    "role": "system",
                    "content": "You are an expert insurance agent designed to assist with coverage queries. Use the provided documents to answer questions about insurance policies. If the information is not available in the documents, respond with 'I don't know'"
                },
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": prompt
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:application/pdf;base64,{self.pdf_data}"
                            }
                        }
                    ]
                }
            ],
        )
        
        return response.choices[0].message.content.replace("$", r"\\$")

## 1.5. Test the Agent Class

Finally, import the `PolicyAgent` class you just created and test it with the same query to ensure it works as expected.

In [ ]:
from policy_agent import PolicyAgent

print("Running Health Insurance Policy Agent")
agent = PolicyAgent()
prompt = "How much would I pay for mental health therapy?"

response = agent.answer_query(prompt)
display(Markdown(response))

## 1.6. Resources

- [Google Gemini API Documentation](https://ai.google.dev/docs)
- [LiteLLM Documentation](https://docs.litellm.ai/docs/)

<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>
</div>
